# MAD-X Simulations

This section is based on the **BL2_TDC** ATF example.

**Hint**: See the [List of predefined simulations in Sirepo](../simulations.rst) for examples of identifiers for different beamlines.

In [ ]:
%run -i ../../../examples/prepare_flyer_env.py

from sirepo_bluesky.sirepo_bluesky import SirepoBluesky
from sirepo_bluesky.madx_flyer import MADXFlyer
from sirepo_bluesky.sirepo_ophyd import create_classes

connection = SirepoBluesky("http://localhost:8000")

data, schema = connection.auth("madx", "00000001")
classes, objects = create_classes(connection.data,
                                  connection=connection,
                                  extra_model_fields=["rpnVariables"])
globals().update(**objects)

madx_flyer = MADXFlyer(connection=connection,
                       root_dir="/tmp/sirepo-bluesky-data",
                       report="elementAnimation250-20")

def madx_plan(parameter=ihq1, value=2.0):
    yield from bps.mv(parameter.value, value)
    return (yield from bp.fly([madx_flyer]))

(uid,) = RE(madx_plan())  # noqa F821
hdr = db[uid]
tbl = hdr.table(stream_name="madx_flyer", fill=True)
print(tbl)

### Get the original data file:

In [ ]:
import os
import tfs

for name, doc in hdr.documents():
    if name == "resource":
        resource_doc = doc
        print(resource_doc)
filename = os.path.join(resource_doc["root"], resource_doc["resource_path"])
df = tfs.read(filename)

In [ ]:
df

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.plot(df["S"], df["BETX"])
plt.plot(df["S"], df["BETY"])
plt.grid()
plt.xlabel("s [m]")
plt.ylabel("betx, bety [m]")

### Screenshot from Sirepo website

![image.png](../../../images/madx_twiss2_file.png)